# Altinha Ball Tracking and Hit Detection

This notebook performs ball tracking and hit detection on altinha (Brazilian footvolley) videos using YOLO object detection.

<a href="https://colab.research.google.com/github/kifjj/altinha-play/blob/main/alta_infer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install Dependencies

Install required packages with specific versions for compatibility.


In [ ]:
!pip install "numpy<2.0" "scipy<1.14" supervision ultralytics "opencv-python-headless<4.12"

## Import Libraries


In [ ]:
import cv2
import numpy as np
import supervision as sv
from ultralytics import YOLO


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 40.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 MB 31.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 25.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.6 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 32.3 MB/s eta 

## Configuration

Set up paths and hyperparameters for ball detection and hit counting.


In [ ]:
# Video and model paths
VIDEO_PATH = '/kaggle/input/alta-videos/altinha-beach-green-mq-13s.mp4'
MODEL_PATH = '/kaggle/input/yolo-ft-2511/pytorch/default/1/altinha_best.pt'
OUTPUT_PATH = '/kaggle/working/altinha-beach-green-mq-BEST_ONLY.mp4'

POSE_MODEL_PATH = '/kaggle/input/yolo11-pose/pytorch/default/1/yolo11n-pose.pt'

# Detection parameters
CONFIDENCE_THRESHOLD = 0.05  # Minimum confidence for initial detection
MIN_CONFIDENCE = 0.08  # Minimum confidence to keep a detection
IOU_NMS = 0.5  # NMS IoU threshold

# Hit detection parameters
MIN_VERTICAL_AMPLITUDE = 3  # Minimum pixels for a valid hit (vertical movement)
MIN_FRAMES_BETWEEN_HITS = 8  # Minimum frames between consecutive hits
GAP_RESET_FRAMES = 30  # Frames without detection before resetting trajectory (1 sec at 30fps)


## Initialize Models and Annotators


In [ ]:
# Load YOLO model
model = YOLO(MODEL_PATH)
model_pose = YOLO(POSE_MODEL_PATH)

# Setup annotators
box_annotator = sv.BoxAnnotator(
    thickness=2,
    color=sv.Color.from_hex("#00FF00")
)

label_annotator = sv.LabelAnnotator(
    text_scale=0.5,
    text_thickness=2,
    text_position=sv.Position.TOP_CENTER,
)


## Helper Functions


In [ ]:
def filter_best_detection(detections, min_confidence):
    """
    Filter detections to keep only the best one (highest confidence).
    
    Args:
        detections: sv.Detections object
        min_confidence: Minimum confidence threshold
        
    Returns:
        Filtered sv.Detections object (empty if no valid detection)
    """
    if len(detections) == 0:
        return detections
    
    best_idx = int(np.argmax(detections.confidence))
    best_conf = float(detections.confidence[best_idx])
    
    if best_conf >= min_confidence:
        return detections[best_idx:best_idx+1]
    
    return detections[0:0]  # Return empty detections


def update_tracking_state(detections, n_frame, last_positions, last_detection_frame, gap_reset_frames):
    """
    Update ball tracking state with new detection.
    
    Args:
        detections: sv.Detections object
        n_frame: Current frame number
        last_positions: List of (frame_idx, y_center) tuples
        last_detection_frame: Last frame with detection
        gap_reset_frames: Max gap before resetting trajectory
        
    Returns:
        tuple: (updated_last_positions, updated_last_detection_frame)
    """
    if len(detections) == 0:
        return last_positions, last_detection_frame
    
    # Check for gap in detections
    if last_detection_frame is not None:
        gap = n_frame - last_detection_frame
        if gap > gap_reset_frames:
            last_positions = []
    
    # Get ball center
    x1, y1, x2, y2 = detections.xyxy[0].tolist()
    y_center = 0.5 * (y1 + y2)
    
    # Update position history (keep last 3)
    last_positions = last_positions.copy()
    last_positions.append((n_frame, y_center))
    if len(last_positions) > 3:
        last_positions.pop(0)
    
    return last_positions, n_frame


def detect_hit(last_positions):
    """
    Detect if a hit occurred based on ball trajectory.
    A hit is detected when the ball reaches a local maximum in y-coordinate (bottom of screen).
    
    Args:
        last_positions: List of (frame_idx, y_center) tuples (last 3 positions)
        
    Returns:
        tuple: (is_hit, frame_idx, y_center, vertical_span) or (False, None, None, None)
    """
    if len(last_positions) != 3:
        return False, None, None, None
    
    (f0, y0), (f1, y1c), (f2, y2c) = last_positions
    
    # Check if middle point is a local maximum (ball at lowest point)
    going_down_then_up = (y0 < y1c) and (y2c < y1c)
    vertical_span = y1c - min(y0, y2c)
    
    if going_down_then_up and vertical_span >= MIN_VERTICAL_AMPLITUDE:
        return True, f1, y1c, vertical_span
    
    return False, None, None, None


def get_pose_keypoints(frame, model_pose):
    """
    Run YOLO-pose model to detect player keypoints in the frame.
    
    Args:
        frame: Video frame to analyze
        model_pose: YOLO pose model
        
    Returns:
        List of poses, each containing keypoints data
    """
    results = model_pose(frame, verbose=False, conf=0.3)[0]
    
    if results.keypoints is None or len(results.keypoints.data) == 0:
        return []
    
    # Extract keypoints data
    # keypoints shape: (num_persons, num_keypoints, 3) where 3 = (x, y, confidence)
    poses = []
    for person_keypoints in results.keypoints.data:
        poses.append(person_keypoints.cpu().numpy())
    
    return poses


def find_closest_player(ball_center, poses):
    """
    Find the player closest to the ball.
    
    Args:
        ball_center: Tuple (x, y) of ball center
        poses: List of pose keypoints arrays
        
    Returns:
        Tuple (player_pose, player_id) or (None, -1) if no players detected
    """
    if not poses:
        return None, -1
    
    ball_x, ball_y = ball_center
    min_distance = float('inf')
    closest_player_idx = -1
    
    for i, pose in enumerate(poses):
        # Calculate player center from valid keypoints
        valid_keypoints = pose[pose[:, 2] > 0.3]  # Filter by confidence > 0.3
        if len(valid_keypoints) == 0:
            continue
        
        player_x = np.mean(valid_keypoints[:, 0])
        player_y = np.mean(valid_keypoints[:, 1])
        
        # Calculate distance to ball
        distance = np.sqrt((ball_x - player_x)**2 + (ball_y - player_y)**2)
        
        if distance < min_distance:
            min_distance = distance
            closest_player_idx = i
    
    if closest_player_idx == -1:
        return None, -1
    
    return poses[closest_player_idx], closest_player_idx


def classify_hit_type(ball_bbox, player_pose):
    """
    Classify hit type (Head/Foot/Unknown) based on ball position and player keypoints.
    
    Args:
        ball_bbox: Ball bounding box in xyxy format
        player_pose: Player keypoint array (17, 3) with (x, y, confidence)
        
    Returns:
        String: 'Head', 'Foot', or 'Unknown'
    """
    if player_pose is None:
        return 'Unknown'
    
    # Calculate ball center
    ball_x = (ball_bbox[0] + ball_bbox[2]) / 2
    ball_y = (ball_bbox[1] + ball_bbox[3]) / 2
    
    # COCO keypoint indices:
    # Head: 0=nose, 1=left_eye, 2=right_eye, 3=left_ear, 4=right_ear
    # Feet: 15=left_ankle, 16=right_ankle
    head_indices = [0, 1, 2, 3, 4]
    foot_indices = [15, 16]
    
    # Calculate distance to head keypoints
    head_distances = []
    for idx in head_indices:
        if idx < len(player_pose) and player_pose[idx, 2] > 0.3:  # Check confidence
            kp_x, kp_y = player_pose[idx, 0], player_pose[idx, 1]
            dist = np.sqrt((ball_x - kp_x)**2 + (ball_y - kp_y)**2)
            head_distances.append(dist)
    
    # Calculate distance to foot keypoints
    foot_distances = []
    for idx in foot_indices:
        if idx < len(player_pose) and player_pose[idx, 2] > 0.3:  # Check confidence
            kp_x, kp_y = player_pose[idx, 0], player_pose[idx, 1]
            dist = np.sqrt((ball_x - kp_x)**2 + (ball_y - kp_y)**2)
            foot_distances.append(dist)
    
    # Determine hit type based on minimum distances
    min_head_dist = min(head_distances) if head_distances else float('inf')
    min_foot_dist = min(foot_distances) if foot_distances else float('inf')
    
    # If both are unavailable
    if min_head_dist == float('inf') and min_foot_dist == float('inf'):
        return 'Unknown'
    
    # Classification with threshold (prefer the closer one with a margin)
    distance_threshold = 80  # pixels - adjust based on video resolution
    
    if min_head_dist < distance_threshold and min_head_dist < min_foot_dist:
        return 'Head'
    elif min_foot_dist < distance_threshold:
        return 'Foot'
    else:
        return 'Unknown'


def check_and_record_hit(last_positions, hit_detections, fps, min_frames_between_hits, frame, model_pose, ball_bbox):
    """
    Check for hit and record it if valid, with hit type classification.
    
    Args:
        last_positions: List of (frame_idx, y_center) tuples
        hit_detections: List of hit metadata dicts
        fps: Video frames per second
        min_frames_between_hits: Minimum frames between consecutive hits
        frame: Current video frame (for pose detection)
        model_pose: YOLO pose model
        ball_bbox: Ball bounding box (xyxy format)
        
    Returns:
        Updated hit_detections list
    """
    is_hit, hit_frame, hit_y, span = detect_hit(last_positions)
    
    if is_hit:
        # Check minimum gap between hits
        last_hit_frame = hit_detections[-1]['frame'] if hit_detections else None
        if not hit_detections or (hit_frame - last_hit_frame) >= min_frames_between_hits:
            # Analyze pose to classify hit type
            poses = get_pose_keypoints(frame, model_pose)
            ball_center = ((ball_bbox[0] + ball_bbox[2]) / 2, (ball_bbox[1] + ball_bbox[3]) / 2)
            player_pose, player_id = find_closest_player(ball_center, poses)
            hit_type = classify_hit_type(ball_bbox, player_pose)
            
            hit_detections = hit_detections.copy()
            hit_detections.append({
                'frame': hit_frame,
                'type': hit_type,
                'player_id': player_id
            })
            t_sec = hit_frame / fps
            print(f"HIT #{len(hit_detections)} at frame {hit_frame} (t={t_sec:.2f}s), Type: {hit_type}, Player: {player_id}, y={hit_y:.1f}, span={span:.1f}")
            return hit_detections
    
    return hit_detections


def annotate_frame(frame, detections, box_annotator, label_annotator, hit_detections):
    """
    Annotate frame with ball detection and hit counter.
    
    Args:
        frame: Video frame to annotate
        detections: sv.Detections object
        box_annotator: Supervision box annotator
        label_annotator: Supervision label annotator
        hit_detections: List of hit metadata dicts
        
    Returns:
        Annotated frame
    """
    annotated_frame = frame.copy()
    
    # Draw ball detection box and label
    if len(detections) > 0:
        conf = float(detections.confidence[0])
        labels = [f"Ball {conf:.2f}"]
        annotated_frame = box_annotator.annotate(
            scene=annotated_frame,
            detections=detections,
        )
        annotated_frame = label_annotator.annotate(
            scene=annotated_frame,
            detections=detections,
            labels=labels,
        )
    
    # Draw hit counter HUD
    annotated_frame = draw_hit_counter(annotated_frame, hit_detections)
    
    return annotated_frame


def draw_hit_counter(frame, hit_detections):
    """
    Draw a hit counter HUD on the frame with breakdown by hit type.
    
    Args:
        frame: Video frame to annotate
        hit_detections: List of hit metadata dicts
        
    Returns:
        Annotated frame
    """
    # Calculate hit counts by type
    total_hits = len(hit_detections)
    head_hits = sum(1 for h in hit_detections if h['type'] == 'Head')
    foot_hits = sum(1 for h in hit_detections if h['type'] == 'Foot')
    unknown_hits = sum(1 for h in hit_detections if h['type'] == 'Unknown')
    
    hit_text = f"Hits: {total_hits} | Head: {head_hits} | Foot: {foot_hits} | Unknown: {unknown_hits}"
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 0.8
    thickness = 2
    
    # Measure text size
    (text_width, text_height), baseline = cv2.getTextSize(
        hit_text, font, font_scale, thickness
    )
    
    # Box position and padding
    pad_x, pad_y = 10, 10
    x1, y1 = 10, 10
    x2 = x1 + text_width + 2 * pad_x
    y2 = y1 + text_height + 2 * pad_y
    
    # Draw filled rectangle
    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), thickness=-1)
    
    # Draw text
    text_x = x1 + pad_x
    text_y = y1 + pad_y + text_height
    cv2.putText(
        frame, hit_text, (text_x, text_y),
        font, font_scale, (0, 255, 0), thickness, cv2.LINE_AA
    )
    
    return frame


In [ ]:
def debug_print_detections(detections, n_frame):
    if len(detections) > 0:
        conf = float(detections.confidence[0])
        x1, y1, x2, y2 = detections.xyxy[0].tolist()


        print(
            f"FRAME {n_frame:4d}: best conf={conf:.3f}, "
            f"bbox=({x1:.1f},{y1:.1f},{x2:.1f},{y2:.1f})"
        )
    else:
        print(f"FRAME {n_frame:4d}: no detection")

## Process Video

Run ball detection and hit counting on the video.


In [ ]:
# Get video info
video_info = sv.VideoInfo.from_video_path(VIDEO_PATH)
fps = video_info.fps
frames_generator = sv.get_video_frames_generator(VIDEO_PATH)

print(f"Processing {VIDEO_PATH}")
print(f"FPS: {fps}, Resolution: {video_info.width}x{video_info.height}\n")

# Initialize tracking state
last_positions = []  # Track last 3 (frame_idx, y_center) positions
hit_detections = []  # List of hit metadata dicts: {'frame': int, 'type': str, 'player_id': int}
last_detection_frame = None  # Last frame with a detection

# Process video
with sv.VideoSink(target_path=OUTPUT_PATH, video_info=video_info) as sink:
    for n_frame, frame in enumerate(frames_generator, start=1):
        # Run YOLO detection
        results = model(
            frame,
            verbose=False,
            conf=CONFIDENCE_THRESHOLD,
            iou=IOU_NMS,
        )[0]
        
        detections = sv.Detections.from_ultralytics(results)
        
        # Filter to keep only best detection
        detections = filter_best_detection(detections, MIN_CONFIDENCE)
        
        debug_print_detections(detections, n_frame)

        

        # Update tracking state
        last_positions, last_detection_frame = update_tracking_state(
            detections, n_frame, last_positions, last_detection_frame, GAP_RESET_FRAMES
        )
        
        # Check for hit and record it
        ball_bbox = detections.xyxy[0] if len(detections) > 0 else None
        if ball_bbox is not None:
            hit_detections = check_and_record_hit(
                last_positions, hit_detections, fps, MIN_FRAMES_BETWEEN_HITS, frame, model_pose, ball_bbox
            )
        
        # Annotate frame with detections and hit counter
        annotated_frame = annotate_frame(
            frame, detections, box_annotator, label_annotator, hit_detections
        )
        
        # Write frame to output video
        sink.write_frame(annotated_frame)

print(f"\n✅ Done! Video saved to {OUTPUT_PATH}")
print(f"📊 Total hits detected: {len(hit_detections)}")


## Results Summary

Display detailed results of hit detection.


In [ ]:
print("=" * 60)
print("HIT DETECTION RESULTS")
print("=" * 60)

# Calculate hit statistics
total_hits = len(hit_detections)
head_hits = sum(1 for h in hit_detections if h['type'] == 'Head')
foot_hits = sum(1 for h in hit_detections if h['type'] == 'Foot')
unknown_hits = sum(1 for h in hit_detections if h['type'] == 'Unknown')

print(f"Total hits/passes detected: {total_hits}")
print(f"  - Head hits: {head_hits} ({100*head_hits/total_hits:.1f}%)" if total_hits > 0 else "  - Head hits: 0")
print(f"  - Foot hits: {foot_hits} ({100*foot_hits/total_hits:.1f}%)" if total_hits > 0 else "  - Foot hits: 0")
print(f"  - Unknown: {unknown_hits} ({100*unknown_hits/total_hits:.1f}%)" if total_hits > 0 else "  - Unknown: 0")

print(f"\nDetailed timestamps:")
print("-" * 60)

for i, hit_data in enumerate(hit_detections, start=1):
    frame_idx = hit_data['frame']
    hit_type = hit_data['type']
    player_id = hit_data['player_id']
    timestamp = frame_idx / fps
    minutes = int(timestamp // 60)
    seconds = timestamp % 60
    print(f"  Hit #{i:2d} | Frame {frame_idx:4d} | {minutes:02d}:{seconds:05.2f} | {hit_type:7s} | Player {player_id}")

print("=" * 60)
